In [16]:
from VPython.Layer import Tensor, Convolution2D, Flatten, Dense, MaxPooling2D
from VPython.NetWork import *
import numpy as np
from VPython.Activator import *

In [17]:
from PIL import Image

Input = Tensor(28*28,1)
out = Dense(neurons=100, activation='logistic', biasUsed=True)(Input)
out = Dense(neurons=100, activation='logistic')(out)
softmaxOut = Dense(neurons=12, activation="softmax")(out)
optimizer = SGD(lr=0.1, decay=1.0, clipvalue=10)

model = Model(input=Input, output=softmaxOut)
model.compileLoss(Cross_Entropy())
model.compileRegular(None)
model.compileOptimizer(optimizer)

In [18]:

class CustomImageDataset:
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        img1_path=os.path.join(self.img_dir,'1')
        self.n = len(os.listdir(img1_path))
        self.image_list=[]
        for idx in range(self.n*12):
            img_path = os.path.join(self.img_dir,str(int(idx/self.n )+ 1)+'/'+str(idx%self.n+1)+".bmp")
            with Image.open(img_path) as im:
                image = im.getdata()
            label = int(idx/self.n )+ 1
            if self.transform:
                image = self.transform(image)
            if self.target_transform:
                label = self.target_transform(label)
            self.image_list.append((image, label))


    def __len__(self):
        return len(self.img_labels)*self.n

    def __getitem__(self, idx):
        return self.image_list[idx]


In [19]:
def transform(x):
    x = np.array(x).reshape(28*28,1)
    return x/255
def target_transform(x):
    res = np.zeros((12,1),dtype=np.float16)
    res[int(x-1)][0] = 1.0
    return res

annotations='博学笃志切问近思自由无用'
annotations=list(annotations)
print(annotations)
imageSet = CustomImageDataset(annotations,'./train/',transform,target_transform)
image, label=imageSet[2212]
image

['博', '学', '笃', '志', '切', '问', '近', '思', '自', '由', '无', '用']


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [20]:
def evaluate(model:Model,Train):
    nn=len(Train)
    acc = 0
    for i in range(nn):
        image, label = Train[i]
        input = model.predict(image)
        if np.argmax(input) == np.argmax(label):
            acc += 1
    acc = acc/nn
    print("acc",acc)
    
evaluate(model,imageSet)


acc 0.08333333333333333


In [21]:

def myPrediction(model,path):
    with Image.open(path) as im:
        image = im.getdata()
    image = np.array(image)/255
    input = model.predict(image)
    return annotations[np.argmax(input)]

print(myPrediction(model,"./train/1/10.bmp"))
print(myPrediction(model,"./train/2/10.bmp"))
print(myPrediction(model,"./train/3/10.bmp"))
print(myPrediction(model,"./train/4/10.bmp"))
print(myPrediction(model,"./train/5/10.bmp"))
print(myPrediction(model,"./train/6/10.bmp"))
print(myPrediction(model,"./train/7/10.bmp"))
print(myPrediction(model,"./train/8/10.bmp"))
print(myPrediction(model,"./train/9/10.bmp"))
print(myPrediction(model,"./train/10/10.bmp"))
print(myPrediction(model,"./train/11/10.bmp"))
print(myPrediction(model,"./train/12/10.bmp"))

思
思
思
思
思
思
思
思
思
思
思
思


In [22]:
def fit(model:Model,Train,Test, epoch:int =1,batch_size: int = 1, log: bool = True):
    nn=len(Train)
    input = 0
    sumloss = 0
    loss = 0
    error = False
    lr = model.optimizer.lr
    iteration = int(nn/batch_size)
    print("iteration",iteration)
    lastLoss = 0
    allLoss = 0.0
    worse = 0
    TrainAcc = 0
    TestAcc = 0
    for i in range(epoch):
        TrainAcc = 0
        TestAcc = 0
        print(i)
        testIndex = list(range(nn))
        np.random.shuffle(testIndex)
        lastLoss = allLoss
        allLoss = 0.0
        for j in range(iteration):
            

            sumloss=0.0
            for k in range(batch_size):
                testIndex0 = testIndex[j*batch_size+k]
                image,label = Train[testIndex0]
                input = model.predict(image)
                derivation, loss = model.lossFunc(f=input, res=label)
                if np.isnan(loss):
                    error = True
                    print("Error")
                    break
                for item in model.layer[-1::-1]:
                    derivation = item.feedBackward(derivation, model.optimizer.clipvalue)
                    derivation = np.array(derivation)
                if np.argmax(input) == np.argmax(label):
                    TrainAcc = TrainAcc + 1
                
                sumloss = sumloss + loss
                allLoss = allLoss + loss
            for item in model.layer:
                if error:
                    item.abort()
                    print("Error")
                    return
                else:
                    item.parameterUpdate(batch_size, lr, model.regularization)
            # if xtest is not None and ytest is not None and step is not None:
        for j in range(len(Test)):
            image,label = Test[j]
            input = model.predict(image)
            if np.argmax(input) == np.argmax(label):
                TestAcc = TestAcc +1
        if allLoss > lastLoss:
            worse = worse+1
        else:
            worse  = 0
        if worse > 3:
            lr = max(lr*0.9,0.001)
        if log:
            print("epoch=%d,loss = %.8f,trainAcc=%.8f,testAcc = %.8f" % (i, allLoss/iteration/batch_size,TrainAcc/nn,TestAcc/len(Test)))
        lr = max(lr * model.optimizer.decay,0.001)
mylist = list(range(620))
rate = 0.1

Train = []
Test = []
for i in range(12):
    np.random.shuffle(mylist)
    for j in range(620):
        if j < 620*rate:
            Test.append(imageSet[i*620+mylist[j]])
        else:
            Train.append(imageSet[i*620+mylist[j]])

print(len(Train))
print(len(Test))
for i in Test:
    print(np.argmax(i[1]))

6696
744
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7

In [23]:

fit(model,Train,Test,100,24)

iteration 279
0
epoch=0,loss = 2.50120964,trainAcc=0.08751493,testAcc = 0.08333333
1
epoch=1,loss = 2.45159670,trainAcc=0.12246117,testAcc = 0.24327957
2
epoch=2,loss = 2.29307622,trainAcc=0.24313023,testAcc = 0.31182796
3
epoch=3,loss = 1.93984487,trainAcc=0.37320789,testAcc = 0.15322581
4
epoch=4,loss = 1.64632583,trainAcc=0.44997013,testAcc = 0.58467742
5
epoch=5,loss = 1.42384747,trainAcc=0.54375747,testAcc = 0.49193548
6
epoch=6,loss = 1.17608859,trainAcc=0.63172043,testAcc = 0.71370968
7
epoch=7,loss = 1.00877937,trainAcc=0.69190562,testAcc = 0.70698925
8
epoch=8,loss = 0.90789327,trainAcc=0.71505376,testAcc = 0.71236559
9
epoch=9,loss = 0.81897334,trainAcc=0.74940263,testAcc = 0.74327957
10
epoch=10,loss = 0.74795583,trainAcc=0.76403823,testAcc = 0.80376344
11
epoch=11,loss = 0.67888353,trainAcc=0.79316010,testAcc = 0.79569892
12
epoch=12,loss = 0.65384386,trainAcc=0.79525090,testAcc = 0.78494624
13
epoch=13,loss = 0.61077451,trainAcc=0.81332139,testAcc = 0.81854839
14
epoch=14,

In [25]:
evaluate(model,Test)
sum =[0]*12
for i in Test:
    sum[np.argmax(i[1])]+=1
print(sum)

acc 0.8951612903225806
[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62]
